# Using EcoFOCIpy to process raw field data

## Cruise ID
{nw16xx, dy17xx, hx}

## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

In [57]:
import yaml
import glob
import pandas as pd

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

In [58]:
sample_data_dir = '/Users/bell/ecoraid/2007/CTDcasts/tn211/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [59]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'from_ecoraid/dgk/' #<- point to cruise and process all files within
nutdatafile = sample_data_dir+'working/DiscreteNutrients/TN211 Nutrient Data.txt' #<- point to cruise and process all files within
cruise_name = 'tn211' #no hyphens
cruise_meta_file = sample_data_dir+'logs/tn211.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpNutsWOCE.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data

bdata = pd.read_csv(datafile+'tn211_nuts.tab','\t+')

/var/folders/vv/blq095kj0xj9nz1v0ffdwfch0000gp/T/ipykernel_48719/2112547197.py:13: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  bdata = pd.read_csv(datafile+'tn211_nuts.tab','\t+')
/var/folders/vv/blq095kj0xj9nz1v0ffdwfch0000gp/T/ipykernel_48719/2112547197.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  bdata = pd.read_csv(datafile+'tn211_nuts.tab','\t+')


In [60]:
cruise_data = {}
for i,groups in bdata.groupby('Cast'):
    cruise_data.update({'ctd'+str(i).zfill(3):pd.DataFrame(groups)})

## Load csv Nutrient File

In [61]:
nut_data = pd.read_csv(nutdatafile,delimiter='\t')
nut_data

,Cast,Niskin,PO4 (uM),PO4 Flag,Sil (uM),Sil Flag,NO3 (uM),NO3 Flag,NO2 (uM),NO2 Flag,NH4 (uM),NH4 Flag
0,1,1,3.021,2,233.8,2,39.2,2,0.02,2,0.00,2
1,1,2,3.023,2,227.9,2,40.0,2,0.01,2,0.01,2
2,1,3,3.038,2,222.6,2,40.4,2,0.00,2,0.00,2
3,1,4,3.074,2,217.6,2,41.1,2,0.01,2,0.02,2
4,1,5,3.112,2,211.9,2,41.9,2,0.00,2,0.02,2
...,...,...,...,...,...,...,...,...,...,...,...,...
906,119,9,1.466,2,29.6,2,15.9,2,0.15,2,0.58,2
907,119,10,1.464,2,29.5,2,15.8,2,0.14,2,0.55,2
908,119,11,1.455,2,27.4,2,15.7,2,0.15,2,0.59,2
909,119,12,1.440,2,25.2,2,15.6,2,0.15,2,0.64,2


## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [62]:
# keep_param = ['bottle','prdm']
keep_param = ['bottle','Depth']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(nut_data[nut_data['Cast']==matchcast],cdata.reset_index()[keep_param],right_on='bottle',left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
    except:
        continue

## Add Deployment meta information

In [63]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [64]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)

## Add institutional meta-information


In [65]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [66]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'Sil (uM)':'SI',
                            'PO4 (uM)':'PO4',
                            'NO2 (uM)':'NO2', 
                            'NO3 (uM)':'NO3',
                            'NH4 (uM)':'NH4',
                            'Sil Flag':'SI_WOCE_FLAG',
                            'PO4 Flag':'PO4_WOCE_FLAG',
                            'NO2 Flag':'NO2_WOCE_FLAG', 
                            'NO3 Flag':'NO3_WOCE_FLAG',
                            'NH4 Flag':'NH4_WOCE_FLAG',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")
    except:
        print(f'Skipping {cast}')

In [67]:
cast= 'ctd001'
cruise_data[cast] = cruise_data[cast].rename(columns={
                    'Sil (uM)':'SI',
                    'PO4 (uM)':'PO4',
                    'NO2 (uM)':'NO2', 
                    'NO3 (uM)':'NO3',
                    'NH4 (uM)':'NH4',
                    'Sil Flag':'SI_WOCE_FLAG',
                    'PO4 Flag':'PO4_WOCE_FLAG',
                    'NO2 Flag':'NO2_WOCE_FLAG', 
                    'NO3 Flag':'NO3_WOCE_FLAG',
                    'NH4 Flag':'NH4_WOCE_FLAG',
                    'Niskin':'BTLID',
                    'prdm':'pressure',
                    'empty':'empty', #this will be ignored
                    'flag':'flag'})

cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                            instrument_yaml=inst_config, 
                            operation_yaml=cruise_config,
                            operation_type='ctd')

cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

#add global attributes
cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

cast = cast.lower().split('d')[-1].split('.')[0]
cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                           filename=cruise_name+'c'+cast.zfill(3)+'_nut.nc',format="NETCDF3_CLASSIC")

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history